#### *Imports Needed*

In [1]:
from pandas.io.html import read_html
import numpy as np
import pandas as pd
import os
import requests
import folium # map rendering library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

* *html to float markdown tables to left*

In [2]:
%%html
<style>
table {float:left}
</style>

## **Part 1 - Get and Prepare Dataset**

* Get data from Wikipedia -  [link here](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)
* Create a dataframe with the collected data
* Remove rows where borough is *Not assigned*
* Join Neibhbourhood with the same Postal Code
* Set *Not assigned* neighbourhood with the borough name
***

#### *1. get html data and convert to a pandas dataframe*

In [63]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = read_html(page, index_col=0,  attrs={"class":"wikitable"})
toronto_postal = wikitables[0]
print(toronto_postal.shape)

(287, 2)


#### *2. cleaning and prepare data*

In [64]:
toronto_postal.Borough.replace('Not assigned',np.nan, inplace=True) #set NaN to not assigned borough
toronto_postal.dropna(inplace=True) #drop nan values
toronto_postal['Neighbourhood'] = toronto_postal.groupby(['Postcode'])['Neighbourhood'].apply(lambda x: ', '.join(x)) #join Neibhbourhood with the same Postal Code
toronto_postal = toronto_postal.loc[~toronto_postal.index.duplicated(keep='first')] #remove duplicated postal codes
toronto_postal.Neighbourhood.replace('Not assigned',toronto_postal.Borough, inplace=True) #replace not assigned neighbourhoods with borough name
print('Shape of dataset: %s'%(str(toronto_postal.shape)))

Shape of dataset: (103, 2)


#### *3. test data*
* **must match:**

| PostalCode | Borough          | Neighbourhood                          |
|------------|------------------|----------------------------------------|
| M7A        | Queen's Park     | Queen's Park                           |
| M5X        | Downtown Toronto | First Canadian Place, Underground city |
| M1C        | Scarborough      | Highland Creek, Rouge Hill, Port Union |



In [19]:
toronto_postal.loc[['M7A','M5X','M1C']] #they match ;)

Borough                           Neighbourhood
Postcode                                                          
M7A           Queen's Park                            Queen's Park
M5X       Downtown Toronto  First Canadian Place, Underground city
M1C            Scarborough  Highland Creek, Rouge Hill, Port Union

In [35]:
#reset index to put PostalCode as a column and save on a variable called toronto
toronto = toronto_postal.reset_index()
print('Shape of dataset: %s'%(str(toronto_postal.shape)))

Shape of dataset: (103, 2)


## **Part 2 - Get location *(lat,lon)* for each neighborhood of Toronto**

* Get *(lat,lon)* for each postcode on dataset
* Save the data on a .csv file to prevent unnecessary Google API calls

***

In [37]:
def get_location(row):
    GOOGLE_MAPS_API_URL = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': '%s, Toronto, Ontario'%row.Postcode,
        'sensor': 'false',
        'region': 'CA',
        'key': os.environ['googleMaps_apiKey']
    }
    req = requests.get(GOOGLE_MAPS_API_URL, params=params)
    res = req.json()
    
    # Use the first result
    result = res['results'][0]
    
    row['lat'] = result['geometry']['location']['lat']
    row['lng'] = result['geometry']['location']['lng']
    return row
   

In [49]:
#toronto = toronto.apply(get_location,axis=1)
toronto.to_csv('toronto.csv',index=False)
toronto.head()

Postcode           Borough                     Neighbourhood        lat  \
0      M3A        North York                         Parkwoods  43.753259   
1      M4A        North York                  Victoria Village  43.725882   
2      M5A  Downtown Toronto                      Harbourfront  43.654260   
3      M6A        North York  Lawrence Heights, Lawrence Manor  43.718518   
4      M7A      Queen's Park                      Queen's Park  43.662301   

         lng  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

## **Part 3 - Neighborhoods Clustering and Data Analysis**

* Get *(lat,lon)* for each postcode on dataset
* 

***

In [3]:
ds = pd.read_csv('toronto.csv',names=['Postcode','Borough','Neighborhood','lat','lng'], skiprows=1)
ds.head()

Postcode           Borough                      Neighborhood        lat  \
0      M3A        North York                         Parkwoods  43.753259   
1      M4A        North York                  Victoria Village  43.725882   
2      M5A  Downtown Toronto                      Harbourfront  43.654260   
3      M6A        North York  Lawrence Heights, Lawrence Manor  43.718518   
4      M7A      Queen's Park                      Queen's Park  43.662301   

         lng  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

In [4]:
#toronto (lat,lon)
latitude, longitude = 43.7, -79.4
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(ds['lat'], ds['lng'], ds['Borough'], ds['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [5]:
CLIENT_ID = os.environ['foursquare_ClientId'] # your Foursquare ID
CLIENT_SECRET = os.environ['foursquare_ClientSecret'] # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
toronto_venues = getNearbyVenues(names=ds['Neighborhood'],
                                   latitudes=ds['lat'],
                                   longitudes=ds['lng']
                                  )
toronto_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2  Victoria Village              43.725882              -79.315572   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1           Variety Store       43.751974       -79.333114   
2  Victoria Village Arena       43.723481       -79.315635   
3             Tim Hortons       43.725517       -79.313103   
4               Portugril       43.725819       -79.312785   

          Venue Category  
0                   Park  
1      Food & Drink Shop  
2           Hockey Arena  
3            Coffee Shop  
4  Portuguese Restaurant

In [13]:
toronto_venues.to_csv('toronto_venues.csv', index=False)
toronto_venues = pd.read_csv('toronto_venues.csv')

In [14]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='',prefix_sep='')
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

Yoga Studio  Accessories Store  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Art Gallery  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0            0                    0                 0                   0   
1            0                    0                 0                   0   
2            0                    0                 0                   0   
3            0                    0                 0                   0   
4            0                    0                 0                   0   

   Auto Garage  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  \
0            0              0          0           0           0       0   
1            0              0          0           0           0       0   
2            0              0          0           0           0       0   
3            0              0          0           0           0       0   
4            0              0          0           0           0       0   

   Bank  Bar  Baseball Field  Basketball Court  Basketball Stadium  Beer Bar  \
0     0    0               0                 0                   0         0   
1     0    0               0                 0                   0         0   
2     0    0               0                 0                   0         0   
3     0    0               0                 0                   0         0   
4     0    0               0                 0                   0         0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
0           0                   0          0       0              0   
1           0                   0          0       0              0   
2           0                   0          0       0              0   
3           0                   0          0       0              0   
4           0                   0          0       0              0   

   Bookstore  Boutique  Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  \
0          0         0               0        0            0                0   
1          0         0               0        0            0                0   
2          0         0               0        0            0                0   
3          0         0               0        0            0                0   
4          0         0               0        0            0                0   

   Burger Joint  Burrito Place  Bus Line  Bus Station  Business Service  \
0             0              0         0            0                 0   
1             0              0         0            0                 0   
2             0              0         0            0                 0   
3             0              0         0            0                 0   
4             0              0         0            0                 0   

   Butcher  Cafeteria  Café  Cajun / Creole Restaurant  Candy Store  \
0        0          0     0                          0            0   
1        0          0     0                          0            0   
2        0          0     0                          0            0   
3        

In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio  \
0                           Adelaide, King, Richmond          0.0   
1                                          Agincourt          0.0   
2  Agincourt North, L'Amoreaux East, Milliken, St...          0.0   
3  Albion Gardens, Beaumond Heights, Humbergate, ...          0.0   
4                             Alderwood, Long Branch          0.0   

   Accessories Store  Airport  Airport Food Court  Airport Gate  \
0                0.0      0.0                 0.0           0.0   
1                0.0      0.0                 0.0           0.0   
2                0.0      0.0                 0.0           0.0   
3                0.0      0.0                 0.0           0.0   
4                0.0      0.0                 0.0           0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.033333   
1             0.0              0.0               0.0             0.000000   
2             0.0              0.0               0.0             0.000000   
3             0.0              0.0               0.0             0.000000   
4             0.0              0.0               0.0             0.000000   

   Art Gallery  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0          0.0                  0.0          0.066667                 0.0   
1          0.0                  0.0          0.000000                 0.0   
2          0.0                  0.0          0.333333                 0.0   
3          0.0                  0.0          0.000000                 0.0   
4          0.0                  0.0          0.000000                 0.0   

   Auto Garage  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  \
0          0.0            0.0        0.0         0.0         0.0     0.0   
1          0.0            0.0        0.0         0.0         0.0     0.0   
2          0.0            0.0        0.0         0.0         0.0     0.0   
3          0.0            0.0        0.0         0.0         0.0     0.0   
4          0.0            0.0        0.0         0.0         0.0     0.0   

   Bank       Bar  Baseball Field  Basketball Court  Basketball Stadium  \
0   0.0  0.033333             0.0               0.0                 0.0   
1   0.0  0.000000             0.0               0.0                 0.0   
2   0.0  0.000000             0.0               0.0                 0.0   
3   0.0  0.000000             0.0               0.0                 0.0   
4   0.0  0.000000             0.0               0.0                 0.0   

   Beer Bar  Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
0       0.0    0.000000                 0.0        0.0     0.0            0.0   
1       0.0    0.000000                 0.0        0.0     0.0            0.0   
2       0.0    0.000000                 0.0        0.0     0.0            0.0   
3       0.0    0.111111                 0.0        0.0     0.0            0.0   
4       0.0    0.000000                 0.0        0.0     0.0            0.0   

   Bookstore  Boutique  Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  \
0        0.0       0.0            0.00      0.0          0.0              0.0   
1        0.0       0.0            0.25      0.0          0.0              0.0   
2        0.0       0.0            0.00      0.0          0.0              0.0   
3        0.0       0.0            0.00      0.0          0.0              0.0   
4        0.0       0.0            0.00      0.0          0.0              0.0   

   Burger Joint  Burrito Place  Bus Line  Bus Station  Business Service  \
0           0.0            0.0       0.0          0.0               0.0   
1           0.0            0.0       0.0          0.0               0.0   
2           0.0            0.0       0.0          0.0               0.0   
3           0.0            0.0       0.0          0.0               0.0   
4           0.0            0.0       0.

In [16]:
toronto_grouped.shape

(100, 232)

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [90]:
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood  \
0                           Adelaide, King, Richmond   
1                                          Agincourt   
2  Agincourt North, L'Amoreaux East, Milliken, St...   
3  Albion Gardens, Beaumond Heights, Humbergate, ...   
4                             Alderwood, Long Branch   

       1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0                       Café      Asian Restaurant           Coffee Shop   
1  Latin American Restaurant                Lounge        Breakfast Spot   
2                 Playground                  Park      Asian Restaurant   
3              Grocery Store            Beer Store          Liquor Store   
4                Pizza Place                   Gym          Skating Rink   

  4th Most Common Venue  
0                 Hotel  
1        Sandwich Place  
2         Women's Store  
3              Pharmacy  
4              Pharmacy

## Clustering the Neiborhoods

In [91]:
from sklearn.cluster import KMeans, AgglomerativeClustering
# set number of clusters
kclusters = 9

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 5, 3, 1, 1, 5, 5, 5, 5, 5], dtype=int32)

In [92]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.fit_predict(toronto_grouped_clustering))

toronto_merged = ds

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Postcode           Borough                      Neighborhood        lat  \
0      M3A        North York                         Parkwoods  43.753259   
1      M4A        North York                  Victoria Village  43.725882   
2      M5A  Downtown Toronto                      Harbourfront  43.654260   
3      M6A        North York  Lawrence Heights, Lawrence Manor  43.718518   
4      M7A      Queen's Park                      Queen's Park  43.662301   

         lng  Cluster Labels 1st Most Common Venue  2nd Most Common Venue  \
0 -79.329656             3.0                  Park      Food & Drink Shop   
1 -79.315572             0.0           Coffee Shop  Portuguese Restaurant   
2 -79.360636             0.0           Coffee Shop                   Park   
3 -79.464763             5.0        Clothing Store      Accessories Store   
4 -79.389494             0.0           Coffee Shop                    Gym   

  3rd Most Common Venue 4th Most Common Venue  
0         Women's Store      Department Store  
1          Hockey Arena          Intersection  
2                Bakery    Mexican Restaurant  
3         Women's Store    Miscellaneous Shop  
4                  Park                 Diner

In [93]:
toronto_merged['Cluster Labels'].values

array([ 3.,  0.,  0.,  5.,  0.,  0.,  7.,  5.,  1.,  5.,  1.,  4.,  6.,
        5.,  5.,  5.,  5.,  5.,  5.,  0.,  5.,  3.,  0.,  5.,  0.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  8.,  5.,  0.,  0.,  5.,  5.,  5.,
        5.,  3.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  0.,  1.,  5.,
       nan,  5.,  5.,  5.,  5.,  2.,  5.,  5.,  5.,  0.,  5.,  1.,  3.,
        5.,  0.,  5.,  0.,  5.,  1.,  5.,  1.,  5.,  5.,  5.,  5.,  1.,
        5.,  5.,  5.,  5.,  1.,  5.,  5.,  3.,  5.,  5.,  5.,  1.,  1.,
        3.,  5.,  1.,  6., nan,  5.,  5.,  0.,  5.,  5.,  2.,  5.])

In [94]:
toronto_merged['Cluster Labels'].replace(np.nan,int(toronto_merged['Cluster Labels'].mean()), inplace=True)

In [95]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [96]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['lat'], toronto_merged['lng'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### 9 Clusters was choosed to give the best clustering separation between neighborhoods

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==0 , toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

Borough  Cluster Labels 1st Most Common Venue  \
1         North York             0.0           Coffee Shop   
2   Downtown Toronto             0.0           Coffee Shop   
4       Queen's Park             0.0           Coffee Shop   
5   Downtown Toronto             0.0           Coffee Shop   
19      East Toronto             0.0           Coffee Shop   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
1   Portuguese Restaurant          Hockey Arena          Intersection   
2                    Park                Bakery    Mexican Restaurant   
4                     Gym                  Park                 Diner   
5                     Gym                  Park                 Diner   
19      Health Food Store                   Pub                 Trail   

   5th Most Common Venue 6th Most Common Venue  7th Most Common Venue  \
1          Women's Store          Dessert Shop   Ethiopian Restaurant   
2         Breakfast Spot                   Pub             Restaurant   
4             Hobby Shop    Seafood Restaurant  Portuguese Restaurant   
5             Hobby Shop    Seafood Restaurant  Portuguese Restaurant   
19      Department Store  Ethiopian Restaurant    Empanada Restaurant   

   8th Most Common Venue        9th Most Common Venue  \
1    Empanada Restaurant            Electronics Store   
2                   Café                 Dessert Shop   
4     Persian Restaurant                    Nightclub   
5     Persian Restaurant                    Nightclub   
19     Electronics Store  Eastern European Restaurant   

         10th Most Common Venue  
1   Eastern European Restaurant  
2             French Restaurant  
4            Mexican Restaurant  
5            Mexican Restaurant  
19                    Drugstore

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==1 , toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
8    East York             1.0  Fast Food Restaurant           Pizza Place   
10  North York             1.0           Pizza Place   Japanese Restaurant   
50  North York             1.0           Pizza Place   Empanada Restaurant   
63        York             1.0         Grocery Store           Pizza Place   
70   Etobicoke             1.0           Pizza Place           Coffee Shop   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
8              Gastropub              Pharmacy          Intersection   
10                   Pub                Bakery         Women's Store   
50      Department Store           Event Space  Ethiopian Restaurant   
63     Convenience Store              Bus Line          Dessert Shop   
70        Discount Store    Chinese Restaurant          Intersection   

        6th Most Common Venue        7th Most Common Venue  \
8                        Bank           Athletics & Sports   
10               Dessert Shop         Ethiopian Restaurant   
50          Electronics Store  Eastern European Restaurant   
63       Ethiopian Restaurant          Empanada Restaurant   
70  Middle Eastern Restaurant               Sandwich Place   

   8th Most Common Venue        9th Most Common Venue  \
8              Pet Store         Gym / Fitness Center   
10   Empanada Restaurant            Electronics Store   
50             Drugstore                   Donut Shop   
63     Electronics Store  Eastern European Restaurant   
70      Department Store            Electronics Store   

         10th Most Common Venue  
8                   Gas Station  
10  Eastern European Restaurant  
50                      Dog Run  
63                    Drugstore  
70  Eastern European Restaurant

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==2 , toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
57   North York             2.0        Baseball Field         Women's Store   
101   Etobicoke             2.0                  Pool        Baseball Field   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
57           Dessert Shop           Event Space  Ethiopian Restaurant   
101         Women's Store          Dessert Shop           Event Space   

    6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
57    Empanada Restaurant     Electronics Store  Eastern European Restaurant   
101  Ethiopian Restaurant   Empanada Restaurant            Electronics Store   

           9th Most Common Venue 10th Most Common Venue  
57                     Drugstore             Donut Shop  
101  Eastern European Restaurant              Drugstore

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==3 , toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0   North York             3.0                  Park     Food & Drink Shop   
21        York             3.0                  Park         Women's Store   
40  North York             3.0               Airport                  Park   
52  North York             3.0                   NaN                   NaN   
64        York             3.0                  Park         Women's Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0          Women's Store      Department Store  Ethiopian Restaurant   
21  Fast Food Restaurant                Market      Department Store   
40         Women's Store          Dessert Shop           Event Space   
52                   NaN                   NaN                   NaN   
64      Department Store           Event Space  Ethiopian Restaurant   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
0    Empanada Restaurant     Electronics Store  Eastern European Restaurant   
21   Empanada Restaurant     Electronics Store  Eastern European Restaurant   
40  Ethiopian Restaurant   Empanada Restaurant            Electronics Store   
52                   NaN                   NaN                          NaN   
64   Empanada Restaurant     Electronics Store  Eastern European Restaurant   

          9th Most Common Venue 10th Most Common Venue  
0                     Drugstore             Donut Shop  
21                    Drugstore             Donut Shop  
40  Eastern European Restaurant              Drugstore  
52                          NaN                    NaN  
64                    Drugstore             Donut Shop

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==4 , toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
11  Etobicoke             4.0         Jewelry Store         Women's Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
11          Dessert Shop           Event Space  Ethiopian Restaurant   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
11   Empanada Restaurant     Electronics Store  Eastern European Restaurant   

   9th Most Common Venue 10th Most Common Venue  
11             Drugstore             Donut Shop

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==5 , toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

Borough  Cluster Labels 1st Most Common Venue  \
3         North York             5.0        Clothing Store   
7         North York             5.0                  Café   
9   Downtown Toronto             5.0                  Café   
13        North York             5.0                   Gym   
14         East York             5.0              Pharmacy   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
3      Accessories Store         Women's Store    Miscellaneous Shop   
7       Basketball Court  Gym / Fitness Center  Caribbean Restaurant   
9            Coffee Shop        Clothing Store  Fast Food Restaurant   
13           Coffee Shop      Asian Restaurant            Beer Store   
14                  Park            Beer Store        Cosmetics Shop   

   5th Most Common Venue   6th Most Common Venue 7th Most Common Venue  \
3               Boutique  Furniture / Home Store   Arts & Crafts Store   
7    Japanese Restaurant                   Diner        Discount Store   
9       Sushi Restaurant             Pizza Place            Steakhouse   
13           Supermarket            Concert Hall        Clothing Store   
14          Skating Rink             Curling Ice    Dim Sum Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
3           Carpet Store           Coffee Shop  Vietnamese Restaurant  
7                Dog Run            Donut Shop          Women's Store  
9                  Plaza   Sporting Goods Shop                    Spa  
13    Chinese Restaurant            Restaurant     Dim Sum Restaurant  
14  Ethiopian Restaurant   Empanada Restaurant      Electronics Store

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==6 , toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
12  Scarborough             6.0                   Bar         Women's Store   
94    Etobicoke             6.0             Drugstore                   Bar   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
12          Dessert Shop           Event Space  Ethiopian Restaurant   
94   Rental Car Location         Women's Store          Dessert Shop   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
12   Empanada Restaurant     Electronics Store  Eastern European Restaurant   
94           Event Space  Ethiopian Restaurant          Empanada Restaurant   

   9th Most Common Venue       10th Most Common Venue  
12             Drugstore                   Donut Shop  
94     Electronics Store  Eastern European Restaurant

In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==7 , toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
6  Scarborough             7.0  Fast Food Restaurant        Farmers Market   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
6           Event Space  Ethiopian Restaurant   Empanada Restaurant   

  6th Most Common Venue        7th Most Common Venue 8th Most Common Venue  \
6     Electronics Store  Eastern European Restaurant             Drugstore   

  9th Most Common Venue 10th Most Common Venue  
6            Donut Shop                Dog Run

In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==8 , toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
32  Scarborough             8.0            Playground         Women's Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
32      Department Store           Event Space  Ethiopian Restaurant   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
32   Empanada Restaurant     Electronics Store  Eastern European Restaurant   

   9th Most Common Venue 10th Most Common Venue  
32             Drugstore             Donut Shop